# Live Sign Language Translator
This notebook will take video input from the user and transform it into a list of photos. It will then find what the asl equiv is to the inputted hand sign 

In [4]:
import numpy as np
import cv2
import mediapipe as mp
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

from matplotlib import pyplot as plt

help(hands)

Help on Hands in module mediapipe.python.solutions.hands object:

class Hands(mediapipe.python.solution_base.SolutionBase)
 |  Hands(static_image_mode=False, max_num_hands=2, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
 |  
 |  MediaPipe Hands.
 |  
 |  MediaPipe Hands processes an RGB image and returns the hand landmarks and
 |  handedness (left v.s. right hand) of each detected hand.
 |  
 |  Note that it determines handedness assuming the input image is mirrored,
 |  i.e., taken with a front-facing/selfie camera (
 |  https://en.wikipedia.org/wiki/Front-facing_camera) with images flipped
 |  horizontally. If that is not the case, use, for instance, cv2.flip(image, 1)
 |  to flip the image first for a correct handedness output.
 |  
 |  Please refer to https://solutions.mediapipe.dev/hands#python-solution-api for
 |  usage examples.
 |  
 |  Method resolution order:
 |      Hands
 |      mediapipe.python.solution_base.SolutionBase
 |      builtins.o

I0000 00:00:1705452792.506298       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2


#### Capture Image input and proccess it

In [ ]:
while True:
    success, image = cap.read()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks != None:
        landmarks = results.multi_hand_landmarks[0].landmark
        
        x_coords = np.array([lm.x for lm in landmarks])
        y_coords = np.array([lm.y for lm in landmarks])
        
        # Calculate bounding box
        margin = 50
        x_min = max(0, int(np.min(x_coords) * image.shape[1]) - margin)
        y_min = max(0, int(np.min(y_coords) * image.shape[0]) - margin)
        x_max = min(image.shape[1], int(np.max(x_coords) * image.shape[1]) + margin)
        y_max = min(image.shape[0], int(np.max(y_coords) * image.shape[0]) + margin)
        
        # Ensure a square bounding box
        length = max(x_max - x_min, y_max - y_min)
        x_max = min(image.shape[1], x_min + length)
        y_max = min(image.shape[0], y_min + length)
        # Crop the image to the bounding box
        cropped_image = image[y_min:y_max, x_min:x_max]
        cropped_image = cv2.resize(cropped_image,(128, 128), interpolation=cv2.INTER_CUBIC)

        for hand_landmarks in results.multi_hand_landmarks:
            
            for id, lm, in enumerate(hand_landmarks.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                
                if id == 20:
                    cv2.circle(image, (cx, cy), 25, (255, 0, 255), cv2.FILLED)
    
                mp_draw.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.imshow("Output", image)
            cv2.imshow("Cropped", cropped_image)
            cv2.waitKey(10)





In [ ]:
print(results.multi_hand_landmarks)